

![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/ER_ICD10_PCS.ipynb)




# **ICD10-PCS coding**

To run this yourself, you will need to upload your license keys to the notebook. Otherwise, you can look at the example outputs at the bottom of the notebook. To upload license keys, open the file explorer on the left side of the screen and upload `workshop_license_keys.json` to the folder that opens.

## 1. Colab Setup

Import license keys

In [ ]:
import os
import json

with open('/content/spark_nlp_for_healthcare.json', 'r') as f:
    license_keys = json.load(f)

license_keys.keys()

secret = license_keys['SECRET']
os.environ['SPARK_NLP_LICENSE'] = license_keys['SPARK_NLP_LICENSE']
os.environ['AWS_ACCESS_KEY_ID'] = license_keys['AWS_ACCESS_KEY_ID']
os.environ['AWS_SECRET_ACCESS_KEY'] = license_keys['AWS_SECRET_ACCESS_KEY']
sparknlp_version = license_keys["PUBLIC_VERSION"]
jsl_version = license_keys["JSL_VERSION"]

print ('SparkNLP Version:', sparknlp_version)
print ('SparkNLP-JSL Version:', jsl_version)

SparkNLP-JSL Verision: 2.5.5


Install dependencies

In [ ]:
# Install Java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
! java -version

# Install pyspark
! pip install --ignore-installed -q pyspark==2.4.4

# Install Spark NLP
! pip install --ignore-installed spark-nlp==$sparknlp_version
! python -m pip install --upgrade spark-nlp-jsl==$jsl_version --extra-index-url https://pypi.johnsnowlabs.com/$secret

Import dependencies into Python

In [ ]:
os.environ['JAVA_HOME'] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ['PATH'] = os.environ['JAVA_HOME'] + "/bin:" + os.environ['PATH']

import pandas as pd
from pyspark.ml import Pipeline
from pyspark.sql import SparkSession
import pyspark.sql.functions as F

import sparknlp
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.base import *
import sparknlp_jsl


Start the Spark session

In [ ]:
spark = sparknlp_jsl.start(secret)

## 2. Select the Entity Resolver model and construct the pipeline

Select the models:

**ICD10 Entity Resolver models:**

1.   **chunkresolve_icd10cm_clinical**
2.   **chunkresolve_icd10cm_diseases_clinical**
3.   **chunkresolve_icd10cm_injuries_clinical**
4.   **chunkresolve_icd10cm_musculoskeletal_clinical**
5.   **chunkresolve_icd10cm_neoplasms_clinical**
6.   **chunkresolve_icd10cm_puerile_clinical**



For more details: https://github.com/JohnSnowLabs/spark-nlp-models#pretrained-models---spark-nlp-for-healthcare

In [ ]:
# Change this to the model you want to use and re-run the cells below.
ER_MODEL_NAME = "chunkresolve_icd10cm_clinical"
NER_MODEL_NAME = "ner_clinical"

Create the pipeline

In [ ]:
document_assembler = DocumentAssembler() \
    .setInputCol('text')\
    .setOutputCol('document')

sentence_detector = SentenceDetector() \
    .setInputCols(['document'])\
    .setOutputCol('sentences')

tokenizer = Tokenizer()\
    .setInputCols(['sentences']) \
    .setOutputCol('tokens')

pos_tagger = PerceptronModel()\
    .pretrained("pos_clinical", "en", "clinical/models") \
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("pos_tags")

dependency_parser = DependencyParserModel()\
    .pretrained("dependency_conllu", "en")\
    .setInputCols(["sentences", "pos_tags", "tokens"])\
    .setOutputCol("dependencies")

embeddings = WordEmbeddingsModel.pretrained('embeddings_clinical', 'en', 'clinical/models')\
    .setInputCols(["sentences", "tokens"])\
    .setOutputCol("embeddings")

clinical_ner_model = NerDLModel().pretrained(NER_MODEL_NAME, 'en', 'clinical/models').setInputCols("sentences", "tokens", "embeddings")\
    .setOutputCol("clinical_ner_tags")   

clinical_ner_chunker = NerConverter()\
    .setInputCols(["sentences", "tokens", "clinical_ner_tags"])\
    .setOutputCol("clinical_ner_chunks")

chunk_embeddings = ChunkEmbeddings()\
    .setInputCols("clinical_ner_chunks", "embeddings")\
    .setOutputCol("chunk_embeddings")

entity_resolver = \
    ChunkEntityResolverModel.pretrained(ER_MODEL_NAME,"en","clinical/models")\
    .setInputCols("tokens","chunk_embeddings").setOutputCol("resolution")

pipeline = Pipeline(stages=[
    document_assembler, 
    sentence_detector,
    tokenizer,
    pos_tagger,
    dependency_parser,
    embeddings,
    clinical_ner_model,
    clinical_ner_chunker,
    chunk_embeddings,
    entity_resolver])

empty_df = spark.createDataFrame([['']]).toDF("text")
pipeline_model = pipeline.fit(empty_df)
light_pipeline = LightPipeline(pipeline_model)

pos_clinical download started this may take some time.
Approximate size to download 1.7 MB
[OK!]
dependency_conllu download started this may take some time.
Approximate size to download 16.6 MB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.8 MB
[OK!]
chunkresolve_icd10cm_clinical download started this may take some time.
Approximate size to download 166.3 MB
[OK!]


## 3. Create example inputs

In [ ]:
# Enter examples as strings in this array
input_list = [
"""She is followed by Dr. X in our office and has a history of severe tricuspid regurgitation with mild elevation and PA pressure. On 05/12/08, preserved left and right ventricular systolic function, aortic sclerosis with apparent mild aortic stenosis, and bi-atrial enlargement. She has previously had a Persantine Myoview nuclear rest-stress test scan completed at ABCD Medical Center in 07/06 that was negative. She has had significant mitral valve regurgitation in the past being moderate, but on the most recent echocardiogram on 05/12/08, that was not felt to be significant. She has a history of hypertension and EKGs in our office show normal sinus rhythm with frequent APCs versus wandering atrial pacemaker. She does have a history of significant hypertension in the past. She has had dizzy spells and denies clearly any true syncope. She has had bradycardia in the past from beta-blocker therapy."""
]

# 4. Run the pipeline

In [ ]:
df = spark.createDataFrame(pd.DataFrame({"text": input_list}))
result = pipeline_model.transform(df)
light_result = light_pipeline.fullAnnotate(input_list[0])

# 5. Visualize

Full Pipeline

In [ ]:
result.select(
    F.explode(
        F.arrays_zip('resolution.metadata', 'resolution.begin' , 'resolution.end', 'resolution.result')
    ).alias('cols')
).select(
    F.expr("cols['0']['token']").alias('token/chunk'),
    F.expr("cols['1']").alias('begin'),
    F.expr("cols['2']").alias('end'),
    F.expr("cols['0']['resolved_text']").alias('resolved_text'),
    F.expr("cols['3']").alias('icd10_code'),
).toPandas()

,token/chunk,begin,end,resolved_text,icd10_code
0,severe tricuspid regurgitation,60,89,Rheumatic mitral insufficiency,I051
1,mild elevation,96,109,"Fever, unspecified",R509
2,PA pressure,115,125,"Hypotension, unspecified",I959
3,aortic sclerosis,197,212,Atherosclerosis of aorta,I700
4,apparent mild aortic stenosis,219,247,Supravalvular aortic stenosis,Q253
5,bi-atrial enlargement,254,274,Cardiomegaly,I517
6,a Persantine Myoview nuclear rest-stress test ...,300,349,Abnormal brain scan,R9402
7,significant mitral valve regurgitation,424,461,Rheumatic mitral insufficiency,I051
8,recent echocardiogram,507,527,Overactive bladder,N3281
9,hypertension,600,611,Postprocedural hypertension,I973


Light Pipeline

In [ ]:
light_result[0]['resolution']

[Annotation(entity, 60, 89, I051, {'chunk': '0', 'all_k_results': 'I051:::Q224:::I070:::I071:::I072:::I078:::I079:::I368:::I369:::Q229:::Q228:::K06023:::A91:::I361:::I360:::I081:::J4550:::I362:::Q232:::P9163:::I082:::I050:::I083:::Q233:::Q243', 'all_k_distances': '0.9940:::1.0824:::1.1134:::1.1476:::1.2213:::1.2486:::1.2501:::1.2660:::1.2737:::1.3088:::1.3107:::1.3111:::1.3254:::1.3384:::1.3416:::1.4567:::1.4592:::1.4806:::1.4985:::1.5086:::1.5130:::1.5236:::1.5349:::1.5433:::1.5887', 'confidence': '0.0561', 'all_k_resolutions': 'Rheumatic mitral insufficiency:::Congenital tricuspid stenosis:::Rheumatic tricuspid stenosis:::Rheumatic tricuspid insufficiency:::Rheumatic tricuspid stenosis and insufficiency:::Other rheumatic tricuspid valve diseases:::Rheumatic tricuspid valve disease, unspecified:::Other nonrheumatic tricuspid valve disorders:::Nonrheumatic tricuspid valve disorder, unspecified:::Congenital malformation of tricuspid valve, unspecified:::Other congenital malformations of